# 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)
### 각 토픽당 가장 중요한 단어 5개를 출력

### 1. 뉴스 데이터에 대한 이해

In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

In [2]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

In [3]:
# 총 11,314개의 뉴스 중 첫번째 뉴스 출력
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [4]:
# 20개의 카테고리 
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


### 2. 텍스트 전처리

In [5]:
# 알파벳을 제외한 구두점, 숫자, 특수 문자를 제거
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [6]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [7]:
# 뉴스 데이터에서 불용어 제거
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [8]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

### 3. 정수 인코딩과 단어 집합 만들기

In [9]:
# 각 단어를 (word_id, word_frequency)의 형태로 변형
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1])    # 수행된 결과에서 두번째 뉴스 출력. 

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [10]:
dictionary[66]

'faith'

In [11]:
len(dictionary)

64281

### 4. LDA 모델 훈련시키기

In [13]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
%time \
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS,\
                                           id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

Wall time: 5min 55s
(0, '0.041*"space" + 0.016*"nasa" + 0.009*"launch" + 0.008*"earth"')
(1, '0.022*"scsi" + 0.014*"tobacco" + 0.014*"printer" + 0.012*"color"')
(2, '0.013*"armenian" + 0.011*"armenians" + 0.010*"israel" + 0.009*"turkish"')
(3, '0.011*"people" + 0.011*"would" + 0.007*"think" + 0.006*"many"')
(4, '0.028*"game" + 0.025*"team" + 0.020*"year" + 0.018*"games"')
(5, '0.014*"information" + 0.009*"mail" + 0.009*"available" + 0.009*"list"')
(6, '0.013*"would" + 0.012*"like" + 0.010*"know" + 0.009*"think"')
(7, '0.014*"jesus" + 0.009*"christian" + 0.008*"would" + 0.008*"bible"')
(8, '0.011*"chip" + 0.011*"encryption" + 0.010*"system" + 0.009*"keys"')
(9, '0.023*"wire" + 0.017*"ground" + 0.016*"neutral" + 0.014*"puck"')
(10, '0.008*"power" + 0.006*"used" + 0.006*"high" + 0.006*"engine"')
(11, '0.016*"windows" + 0.015*"file" + 0.014*"window" + 0.013*"files"')
(12, '0.017*"jpeg" + 0.012*"uuencode" + 0.007*"guidelines" + 0.006*"john"')
(13, '0.036*"file" + 0.034*"output" + 0.032*"ent

### 5. LDA 시각화 하기

In [ ]:
"""
import pyLDAvis.gensim
%matplotlib inline
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)
"""

### 6. 문서 별 토픽 분포 보기

In [14]:
# 상위 5개의 문서에 대해서만 토픽 분포를 확인
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(2, 0.29295012), (3, 0.19900966), (6, 0.15044968), (7, 0.1357394), (8, 0.053010184), (10, 0.042731177), (14, 0.015468675), (17, 0.029447285), (19, 0.07214891)]
1 번째 문서의 topic 비율은 [(3, 0.10047443), (6, 0.19580537), (7, 0.34584194), (10, 0.040432308), (12, 0.054316122), (18, 0.24469061)]
2 번째 문서의 topic 비율은 [(2, 0.23781997), (3, 0.13287808), (6, 0.30617908), (7, 0.31019595)]
3 번째 문서의 topic 비율은 [(3, 0.061197326), (6, 0.1789664), (8, 0.59483147), (10, 0.08290129), (12, 0.07018236)]
4 번째 문서의 topic 비율은 [(4, 0.21355993), (6, 0.3384961), (12, 0.3489174), (13, 0.06938926)]


In [22]:
def make_topic_table_per_doc(ldamodel, corpus, texts):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [24]:
topic_table = make_topic_table_per_doc(ldamodel, corpus, tokenized_doc)
topic_table = topic_table.reset_index()
topic_table.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topic_table[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,2.0,0.2929,"[(2, 0.2929493), (3, 0.19900009), (6, 0.150440..."
1,1,7.0,0.3458,"[(3, 0.100675434), (6, 0.1955454), (7, 0.34582..."
2,2,7.0,0.3102,"[(2, 0.23782076), (3, 0.13286923), (6, 0.30618..."
3,3,8.0,0.5949,"[(3, 0.061086643), (6, 0.1791152), (8, 0.59487..."
4,4,12.0,0.3489,"[(4, 0.21355851), (6, 0.33849853), (12, 0.3489..."
5,5,7.0,0.7417,"[(7, 0.7417126), (12, 0.058767177), (18, 0.160..."
6,6,3.0,0.3175,"[(3, 0.31747434), (6, 0.1430436), (11, 0.08233..."
7,7,6.0,0.4179,"[(3, 0.18095222), (6, 0.41792268), (7, 0.19717..."
8,8,6.0,0.5565,"[(3, 0.26006368), (6, 0.55646414), (18, 0.1583..."
9,9,18.0,0.4230,"[(2, 0.036640637), (6, 0.18301511), (8, 0.0590..."
